In [1]:
import calendar
import time

import pygsheets as pygs
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

from config import GS_ACCOUNT_FILE, RESERVATIONS_FOLDER, TEMPLATE_FILE, OWNER_ACCOUNT, RU_MONTH_NAMES


gs_client = pygs.authorize(service_account_file=GS_ACCOUNT_FILE)

In [3]:
class Spreadsheet:

    def __init__(self, year: int, month: int):
        self.year = year
        self.month = month
        self.monthname = RU_MONTH_NAMES[month - 1]
        self.ndays = calendar.monthrange(year, month)[1]
        self.filename = f'MonNom - Бронь - {self.monthname} {year}'
        self.file_id = self.metadata.get('id')

    def create(self):

        if self.file_id:
            err = f"Spreadsheet '{self.filename}' already exists"
            raise FileExistsError(err)

        spreadsheet = gs_client.create(
            self.filename, TEMPLATE_FILE, RESERVATIONS_FOLDER)
        self._add_worksheets(spreadsheet)

        self.file_id = self.metadata['id']

    def delete(self):
        if not self.file_id:
            err = f"Spreadsheet '{self.filename}' doesn't exist."
            raise FileNotFoundError(err)

        gs_client.drive.delete(self.file_id)
        self.file_id = ''

    @property
    def metadata(self) -> dict:
        file_list = gs_client.drive.list(q=f"name='{self.filename}'")

        if len(file_list) > 1:
            err = f"Found multiple files named '{self.filename}': \n{file_list}"
            raise ValueError(err)

        return file_list[0] if file_list else {}

    @property
    def worksheet_names(self) -> list[str]:
        month = str(self.month).zfill(2)
        year = str(self.year)[-2:]
        dates = [str(i).zfill(2) for i in range(1, self.ndays + 1)]
        return [f"{date}.{month}.{year}" for date in dates]

    def _add_worksheets(self, sh: pygs.Spreadsheet):

        template = sh.worksheet()

        pbar = tqdm_notebook(self.worksheet_names, leave=False)
        pbar.set_description('Adding worksheets')
        for name in pbar:
            ws = sh.add_worksheet(name, src_worksheet=template)
            self._create_protected_range(ws)
            time.sleep(2.5)  # to comply with Google API limits

        sh.del_worksheet(template)

    @staticmethod
    def _create_protected_range(ws: pygs.Worksheet):
        """ Protected ranges are not copied from template """
        r = ws.create_protected_range(start='A1', end='Z2')
        r.editors = ('users', OWNER_ACCOUNT)